# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

L. Xie  ->  Z.-L. Xie  |  ['L. Xie']
J. Matharu  ->  J. Matharu  |  ['J. Matharu']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
K. Kreckel  ->  K. Kreckel  |  ['K. Kreckel']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


Arxiv has 81 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.15789


extracting tarball to tmp_2511.15789... done.
Retrieving document from  https://arxiv.org/e-print/2511.15792


extracting tarball to tmp_2511.15792... done.


J. Matharu  ->  J. Matharu  |  ['J. Matharu']


...

extracting tarball to tmp_2511.15835...

 done.
Retrieving document from  https://arxiv.org/e-print/2511.16323


extracting tarball to tmp_2511.16323... done.
Retrieving document from  https://arxiv.org/e-print/2511.16437


extracting tarball to tmp_2511.16437...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.15792-b31b1b.svg)](https://arxiv.org/abs/2511.15792) | **A first look at a complete view of spatially resolved star formation at 1<z<1.8 with JWST NGDEEP+FRESCO slitless spectroscopy**  |
|| <mark>J. Matharu</mark>, et al. |
|*Appeared on*| *2025-11-21*|
|*Comments*| *7 pages, 7 figures, to be submitted to A&A*|
|**Abstract**|            [abridged] The previously inaccessible star formation tracer Pa$\alpha$ can now be spatially resolved by JWST NIRCam slitless spectroscopy in distant galaxies up to cosmic noon. In the first study of its kind, we combine JWST NGDEEP NIRISS and FRESCO NIRCam slitless spectroscopy to provide the first direct comparison of spatially resolved dust-obscured (traced by Pa$\alpha$) versus unobscured (traced by H$\alpha$) star formation across the main sequence. We stack Pa$\alpha$ and H$\alpha$ emission-line maps, along with stellar continuum images at both wavelengths of 31 galaxies at 1<z<1.8 in three bins of stellar mass. Surface brightness profiles are measured and equivalent width (EW) profiles computed. Increasing Pa$\alpha$ and H$\alpha$ EW profiles with galactocentric radius across all stellar masses probed provide direct evidence for the inside-out growth of galaxies both via dust-obscured and unobscured star formation for the first time. For galaxies on the main sequence, a weakly positive ($0.1\pm0.1$) Pa$\alpha$/H$\alpha$ line profile as a function of radius is found at $8.8\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<9.9$ with a negative ($-0.4\pm0.1$) Pa$\alpha$/H$\alpha$ line profile found at $9.9\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<11.0$. Low mass galaxies ($7.7\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<8.8$) with high sSFRs are found to have a negative ($-0.5\pm0.1$) Pa$\alpha$/H$\alpha$ line profile gradient. Our results demonstrate that while inside-out growth via star formation is ubiquitous across the main sequence just after cosmic noon, centrally concentrated dust attenuation is not. Along with other recent work in the literature, our findings motivate future studies of resolved SFR profiles in large samples of individual cosmic noon galaxies across the main sequence, to understand the intrinsic scatter in spatially resolved star formation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.15789-b31b1b.svg)](https://arxiv.org/abs/2511.15789) | **Do z>6 quasars reside in protoclusters?**  |
|| F. Fontanot, et al. -- incl., <mark>L. Xie</mark> |
|*Appeared on*| *2025-11-21*|
|*Comments*| *9 Pages, 7 Figures, Submitted to A&A*|
|**Abstract**|            We discuss the properties of a sample of z>6 bright (bolometric luminosity L$_{\rm bolo}$>10$^{46.25}$ erg/s) Quasars drawn from a realization of the GAlaxy Evolution and Assembly (GAEA) model coupled with the Planck Millennium Simulation. We focus on the properties and environment of host galaxies, and their evolution down to z=0, with the aim of assessing how well the bright high redshift QSOs population traces the progenitors of most massive haloes in the local Universe. Our results show that at z>6 bright QSOs live in a variety of environments, and that secular processes like disc instability are responsible for triggering roughly the same number of QSOs as galaxy mergers. Half of cubic (7.5 $h^{-1}$ cMpc size) mock fields built around these high-z QSOs include other active galaxies (with L$_{\rm bolo}$>10$^{44}$ erg/s) in sizeable number, the other host galaxies being relatively isolated. The large field-to-field variance in the the number of companions (both active and non-active) recently reported from JWST observations is fairly well reproduced by GAEA predictions. Descendants of host galaxies at z=0 cover a wide range of physical properties and environments with only a small fraction of the hosts of high-z QSOs ending up in massive galaxy clusters. Viceversa, GAEA predicts that only a small fraction of Bright Central Galaxies have a bright z>6 QSOs among their progenitors. Our results suggest that luminous high-z QSO loosely trace the progenitors of low-z galaxy clusters, and that additional information about the environment of high-z QSOs are required to identify the most promising proto-cluster candidates.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.15835-b31b1b.svg)](https://arxiv.org/abs/2511.15835) | **Methane on the temperate exo-Saturn TOI-199b**  |
|| A. Bello-Arufe, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-21*|
|*Comments*| *23 pages, 16 figures, submitted to AAS Journals*|
|**Abstract**|            Temperate ($T_{\rm eq}<400$ K) gas giants represent an unexplored frontier in exoplanet atmospheric spectroscopy. Orbiting a G-type star every ~100 days, the Saturn-mass exoplanet TOI-199 b ($T_{\rm eq}=350$ K) is one of the most favorable low-temperature gas giants for atmospheric study. Here, we present its transmission spectrum from a single transit observed with JWST's NIRSpec G395M mode. Despite lower-than-nominal precision due to a pointing misalignment, Bayesian retrievals reveal the presence of CH$_4$ (Bayes factor of $\sim$700 in a cloudy atmosphere), corresponding to a metallicity of $\rm{C/H}=13^{+78}_{-12}\times$ solar, although the absence of detectable CO and CO$_2$ disfavors metallicities $\gtrsim50\times$ solar. We also tested several haze prescriptions (Titan-like tholin, soot, and water-rich tholin), but the preference for these models is weak (Bayes factors of $\sim 2$ relative to the clear case). The spectrum also shows an increase in transit depth near 3 $\mu$m, which our self-consistent models attribute to either NH$_3$ or, less likely, HCN. Follow-up observations will distinguish between these species, helping determine the planet's vertical mixing regime. The TOI-199 system exhibits strong transit timing variations (TTVs) due to an outer non-transiting giant planet. For planet c, our TTV analysis reduces its mass uncertainty by 50% and prefers a slightly longer orbital period (still within the conservative habitable zone) and higher eccentricity relative to previous studies. TOI-199 b serves as the first data point for studying clouds and hazes in temperate gas giants. The detection of methane supports the emerging trend that temperate low-molecular-weight atmospheres display spectral features in transmission.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.16437-b31b1b.svg)](https://arxiv.org/abs/2511.16437) | **Aql X-1 from dawn 'til dusk: the early rise, fast state transition and decay of its 2024 outburst**  |
|| A. Marino, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-11-21*|
|*Comments*| *13 pages, 9 figures, submitted to A&A*|
|**Abstract**|            Transient Low-Mass X-ray Binaries (LMXBs) are usually first detected by all-sky X-ray monitors when they enter new outbursts, typically at X-ray luminosities above $\sim$10$^{36}$ erg/s. Observations of these sources during the early rise of the outbursts have so far been very limited. However, the launch of the Einstein Probe (EP) has greatly improved our ability to detect fainter X-ray activity, unlocking access to the outburst early rise. In September 2024, EP detected the early onset of a new outburst from the neutron star LMXB Aql X-1, catching the source at a luminosity below 10$^{35}$ erg/s. In this paper we present results from a comprehensive, multi-wavelength campaign of this event, combining data from EP, NICER, NuSTAR, Swift and Las Cumbres Observatory covering the full outburst from its early rise through its decay. By comparing X-ray and optical light curves obtained with Las Cumbres Observatory during the initial rise, we show that the start of the X-ray emission lagged the optical rise by, at most, 3 days. Time-resolved X-ray spectroscopy revealed how the geometry and the physical properties of the accretion flow evolve during this early stage of the outburst, as well as at higher luminosities as the source transitioned through the canonical X-ray spectral states - hard, intermediate and soft. These data show that the source underwent a very rapid, about 12-h long, transition from the hard to the soft state about two weeks after the optical onset of the outburst. The evolution of the temperature and physical sizes of both the inner region of the disk and a black body near the NS surface suggest that at the state transition, a boundary and spreading layer likely formed. We discuss these results in the context of time-scales for outburst evolution and state transitions in accreting neutron stars and black holes.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.16323-b31b1b.svg)](https://arxiv.org/abs/2511.16323) | **Can Wolf-Rayet stars be the missing ingredient to explain high-z He II ionizing radiation?**  |
|| G. González-Torà, et al. -- incl., <mark>K. Kreckel</mark> |
|*Appeared on*| *2025-11-21*|
|*Comments*| *3 pages, 2 figures, to appear at the Proceedings of the IAUS402: Massive Stars Across Redshifts in the Era of JWST and Large-Scale Surveys*|
|**Abstract**|            Classical Wolf-Rayet (WR) stars are hot, massive stars with depleted hydrogen. At low metallicities (Z), WN3-type WR stars have relatively thin winds and are major sources of ionizing flux. The detection of high-ionization emission lines in high-redshift ($z$) galaxies as well as nearby low-Z dwarf galaxies raises questions about the origin of He II ionizing radiation and its role in galaxy evolution, as stellar population models fail to reproduce the required fluxes. Low-Z WN3 stars may provide the missing contribution but are easily hidden in integrated light. Using the Local Volume Mapper, we compare resolved optical spectra of SMC WN3 stars with integrated regions, focusing on the broad He II $\lambda4686\,Å$ line. We find stellar emission diluted within nebular regions, becoming undetectable when integrating over areas larger than 24 pc. Nonetheless, these stars emit enough ionizing photons to explain observed He II nebular emission, being strong candidates for the He II ionizing sources in low-Z and high-$z$ galaxies.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Max' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2511.15792/./SFMS.png', 'tmp_2511.15792/./PSF_profiles_Jy_smaller.png', 'tmp_2511.15792/./stacks_ergs_vertical.png']
copying  tmp_2511.15792/./SFMS.png to _build/html/
copying  tmp_2511.15792/./PSF_profiles_Jy_smaller.png to _build/html/
copying  tmp_2511.15792/./stacks_ergs_vertical.png to _build/html/
exported in  _build/html/2511.15792.md
    + _build/html/tmp_2511.15792/./SFMS.png
    + _build/html/tmp_2511.15792/./PSF_profiles_Jy_smaller.png
    + _build/html/tmp_2511.15792/./stacks_ergs_vertical.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ha}{H\upalpha}$
$\newcommand{\hb}{H\upbeta}$
$\newcommand{\pa}{Pa\upalpha}$
$\newcommand{\jwst}{JWST}$
$\newcommand{\hst}{HST}$
$\newcommand{\editR}{\textbf{}}$
$\begin{document}$
$   \title{A first look at a complete view of spatially resolved star formation at 1<z<1.8 with JWST NGDEEP+FRESCO slitless spectroscopy}$
$    \titlerunning{Spatially resolved Paschen-alpha vs. H-alpha}$
$    \authorrunning{Matharu et al.}$
$   \author{Jasleen Matharu$
$           \inst{1}\fnmsep$
$           \inst{2}\fnmsep$
$           \inst{3}\thanks{{Corresponding author; \email{jamatharu@mpia.de}}}\orcidlink{0000-0002-7547-3385},$
$           Lu Shen$
$           \inst{4}\fnmsep$
$           \inst{5},$
$           Irene Shivaei$
$           \inst{6},$
$           Pascal A. Oesch$
$           \inst{1}\fnmsep$
$           \inst{2}\fnmsep$
$           \inst{7},$
$           Casey Papovich$
$           \inst{4}\fnmsep$
$           \inst{5},$
$           Gabriel Brammer$
$           \inst{1}\fnmsep$
$           \inst{2}\orcidlink{0000-0003-2680-005X},$
$           Naveen A. Reddy\inst{8}\orcidlink{0000-0001-9687-4973},$
$           Yingjie Cheng$
$           \inst{9}\fnmsep$
$           \inst{10}\orcidlink{0000-0001-8551-071X},$
$           Pieter van Dokkum\inst{11}\orcidlink{0000-0002-8282-9888},$
$           Steven L. Finkelstein$
$           \inst{12}\fnmsep$
$           \inst{13}\orcidlink{0000-0001-8519-1130},$
$           Nimish P. Hathi\inst{14}\orcidlink{0000-0001-6145-5090},$
$           Jeyhan S. Kartaltepe\inst{15}\orcidlink{0000-0001-9187-3605},$
$           Anton M. Koekemoer\inst{14}\orcidlink{0000-0002-6610-2048},$
$           Jorryt Matthee\inst{16}\orcidlink{0000-0003-2871-127X},$
$           Nor Pirzkal\inst{14}\orcidlink{0000-0003-3382-5941},$
$           Stephen M.~Wilkins\inst{17}\fnmsep$
$           \inst{18}\orcidlink{0000-0003-3903-6935},$
$           Michael A. Wozniak\inst{8}\orcidlink{0000-0002-1033-3656}, and$
$           Mengyuan Xiao\inst{7}\orcidlink{0000-0003-1207-5344}}$
$   \institute{{Cosmic Dawn Center, Copenhagen, Denmark}$
$        \and{Niels Bohr Insitute, University of Copenhagen, Jagtvej 128, 2200 Copenhagen, Denmark}$
$        \and{Max-Planck-Institut für Astronomie, Königstuhl 17, D-69117 Heidelberg, Germany}$
$        \and{Department of Physics and Astronomy, Texas A\&M University, College Station, TX, 77843-4242, USA}$
$        \and{George P. and Cynthia Woods Mitchell Institute for Fundamental Physics and Astronomy, Texas A\&M University, College Station, TX, 77845-4242, USA}$
$        \and{Centro de Astrobiología (CAB), CSIC-INTA, Carretera de Ajalvir km 4, Torrejón de Ardoz, 28850, Madrid, Spain}$
$        \and{Department of Astronomy, University of Geneva, Chemin Pegasi 51, 1290 Versoix, Switzerland}$
$        \and{Department of Physics and Astronomy, University of California,$
$Riverside, 900 University Avenue, Riverside, CA 92521, USA}$
$        \and{Department of Astronomy, University of Washington, Seattle, WA 98195, USA}$
$        \and{University of Massachusetts Amherst, 710 North Pleasant Street, Amherst, MA 01003-9305, USA}$
$        \and{Astronomy Department, Yale University, 52 Hillhouse Ave,$
$New Haven, CT 06511, USA}$
$        \and{Department of Astronomy, The University of Texas at Austin, Austin, TX, USA}$
$        \and{Cosmic Frontier Center, The University of Texas at Austin, Austin, TX 78712, USA}$
$        \and{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}$
$        \and{Laboratory for Multiwavelength Astrophysics, School of Physics and Astronomy, Rochester Institute of Technology, 84 Lomb Memorial Drive, Rochester, NY 14623, USA}$
$        \and{Institute of Science and Technology Austria (ISTA), Am Campus 1, A-3400 Klosterneuburg, Austria}$
$        \and{Astronomy Centre, University of Sussex, Falmer, Brighton BN1 9QH, UK}$
$        \and{Institute of Space Sciences and Astronomy, University of Malta, Msida MSD 2080, Malta}}$
$   \date{Received XXX; accepted XXX}$
$  \abstract{The previously inaccessible star formation tracer Paschen-Alpha ({\pa}) can now be spatially resolved by JWST NIRCam slitless spectroscopy in distant galaxies up to cosmic noon. In the first study of its kind, we combine JWST NGDEEP NIRISS and FRESCO NIRCam slitless spectroscopy to new the first direct comparison of spatially resolved dust-obscured (traced by {\pa}) versus unobscured (traced by {\ha}) star formation across the main sequence. We stack {\pa} and {\ha} emission-line maps,  along with stellar continuum images at both wavelengths of 31 galaxies at 1<z<1.8 in three bins of stellar mass spanning 7.7\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<11. Surface brightness profiles are measured and equivalent width (EW) profiles computed. Increasing {\pa} and {\ha} EW profiles with galactocentric radius across all stellar masses probed new direct evidence for the inside-out growth of galaxies both via dust-obscured and unobscured star formation for the first time. For galaxies predominantly on the main sequence (\mathrm{log}(M_{*}/\mathrm{M}_{\odot})\geqslant8.8), a weakly positive (0.1\pm0.1) {\pa}/{\ha} line profile gradient as a function of radius is found at 8.8\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<9.9 with a negative (-0.4\pm0.1) {\pa}/{\ha} line profile gradient as a function of radius found at the highest stellar masses of 9.9\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<11.0. Low mass galaxies (7.7\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<8.8) with predominantly high specific star formation rates (sSFRs) relative to the main sequence are also found to have a negative (-0.5\pm0.1) {\pa}/{\ha} line profile gradient as a function of radius.$
$   Our results demonstrate that while inside-out growth via star formation is ubiquitous across the main sequence just after cosmic noon, centrally concentrated dust attenuation is not. Along with other recent work in the literature, our findings now motivate future studies of resolved SFR and dust attenuation profiles in large samples of individual cosmic noon galaxies across the main sequence, to understand the intrinsic scatter in spatially resolved star formation.}$
$   \keywords{Galaxies: evolution --$
$                Galaxies: high-redshift --$
$                Galaxies: star formation$
$               -- Galaxies: stellar content -- Galaxies: structure}$
$   \maketitle$
$\section{Introduction}$
$Understanding how galaxies build up their stellar mass over cosmic time is a central goal in observational astrophysics, providing critical insight into how galaxies develop their structure. Tracking ongoing star formation in galaxies is therefore key to revealing the physical processes that regulate galaxy growth. Spatially resolving star formation in galaxies is the only way to determine {\it where} star formation is occurring in galaxies, thereby revealing how galaxies assemble their stellar mass and evolve.$
$Multiple observational tracers spanning from the X-ray to radio regime of the electromagnetic spectrum can be used to trace ongoing star formation in galaxies over various timescales. Of these, emission-line tracers new a near-instantaneous measure of star formation due to their capability of tracing emission from the ionised gas surrounding massive young stars ( (Kennicutt2012)  and references therein). The {\ha} emission line has been a popular choice for observing star formation in both nearby and distant galaxies due to its easy access in the rest-frame optical and high intrinsic brightness. Spatially resolved {\ha} emission-line maps have revealed that from the end of reionisation (z\sim5.3) to z\sim0.5, galaxies grow "inside-out" by forming new stars at larger and larger galactocentric radii over time  (Nelson2012, [ and Nelson 2016](),  Wilman2020, [Matharu, Papovich and Simons 2022](), [Matharu, Nelson and Brammer 2024](), [Shen, Papovich and Matharu 2024](), [Danhaive, Tacchella and McClymont 2025]())  but may be experiencing centralised bursty star formation into the epoch of reionisation  ([Stephenson, Stott and Pirie 2025]()) .$
$Despite the conveniences of using {\ha} as a star formation tracer, its susceptibility to dust attenuation leads to a systematic underestimation of star formation rates (e.g.  (Whitaker2014,  Shivaei2016) ). In cases of low to moderate dust attenuation, {\ha}-derived star formation rates (SFRs) can be corrected using the Balmer decrement, whereby the {\ha}/{\hb} flux ratio together with an assumed dust attenuation curve can be used to quantify the amount of dust attenuation towards star-forming regions  (Calzetti1999, [ and Osterbrock 2006]()) . Integrated Balmer decrements increase with stellar mass and SFR  (Wild2011, Dominguez2012, Momcheva2013, [Price, Kriek and Brammer 2014](), Reddy2015, [Shivaei, Reddy and Shapley 2015](), [Pirzkal, Rothberg and Papovich 2024](), [Sandles and Maiolino 2024]())  but remain constant with redshift: a trend that is difficult to physically explain$
$ ([Shapley, Sanders and Salim 2022](), [Shapley, et. al 2023]()) . The few high redshift (z\geqslant1) studies that exist on spatially resolved Balmer decrements of typical star-forming galaxies find no shape evolution in the dust attenuation profiles with redshift at fixed stellar mass to help explain this trend, with predominantly centrally concentrated negative profiles found  (Nelson2016a, [Matharu, Muzzin and Sarrouh 2023]()) .$
$A gold standard alternative for obtaining a complete view of star formation in distant galaxies lies with the Paschen-Alpha ({\pa}) emission line. By virtue of being in the rest-frame near infra-red (NIR), it is insensitive to dust attenuation. {\pa} emission is therefore capable of tracing star formation in more heavily dust-obscured regions that would be optically thick to Balmer lines  (Kennicutt2012) . Telluric OH emission, thermal emission, and its intrinsic faintness have made this line inaccessible for ground-based observatories. The Spitzer Space Telesecope could access {\pa} in high redshift galaxies, but only in rare cases of strong gravitational lensing  ([Papovich, Rudnick and Rigby 2009](), [Shipley, et. al 2016]()) . Its rest-frame wavelength (1.875\mu\mathrm{m}) lies beyond the WFC3/IR wavelength regime covered by the Hubble Space Telescope (HST), making it inaccessible from space as well. Now, with the launch of {\it} JWST, the {\pa} emission line has become accessible in multiple observational modes out to cosmic noon (NIRSpec slit spectroscopy:  ([Reddy, et. al 2023]()) , NIRCam medium-band imaging:  ([Williams, Tacchella and Maseda 2023](), [Suess, Weaver and Price 2024]())  and NIRCam slitless spectroscopy:  ([Liu, Morishita and Kodama 2024](), [Neufeld and Asali 2024]()) ). So far there is evidence to suggest that {\pa}-derived SFRs could be \sim25\% higher than those derived from Balmer lines  ([Reddy, et. al 2023]())  with SFR({\pa}/{\ha}) increasing with stellar mass  ([Liu, Morishita and Kodama 2024]()) . At 1<z<1.6, the {\pa} versus stellar continuum effective radius is positive and increases with stellar mass, providing the first direct evidence for the inside-out growth of galaxies via dust-obscured star formation  ([Liu, Morishita and Kodama 2024]()) .$
$In this paper, we new the first direct spatially resolved comparison of unobscured versus dust-obscured star formation using {\ha} and {\pa} emission-line maps for the same galaxies at 1<z<1.8 residing in the Hubble Ultra Deep Field (HUDF) using {\jwst} NIRISS and NIRCam slitless spectroscopy from the Next Generation Deep Extragalactic Exploratory Public Survey (NGDEEP,  ([Bagley, Pirzkal and Finkelstein 2024]()) ) and the First Reionization Epoch Spectroscopically Complete Observations (FRESCO,  ([Oesch, Brammer and Naidu 2023]()) ). Our sample is described in Section~\ref{sec:sample} and our unique methodology for enabling direct comparisons between NIRCam and NIRISS slitless spectroscopy is detailed in Section~\ref{sec:methodology}. We outline our results in Section~\ref{sec:results} and discuss their physical interpretations in Section~\ref{sec:discussion}. A summary of our findings is given in Section~\ref{sec:summary}.$
$All magnitudes quoted are in the AB system, logarithms are in base 10, and we assume a \LambdaCDM cosmology with \Omega_{m}=0.307, \Omega_{\Lambda}=0.693, and H_{0}=67.7~kms^{-1}~Mpc^{-1}  (Planck2015) .$
$\section{Sample}$
$\label{sec:sample}$
$   \begin{figure}$
$   \centering$
$   \includegraphics[width=\columnwidth]{footprint.pdf}$
$      \caption{The FRESCO and NGDEEP NIRISS observing footprints on the sky.}$
$         \label{fig:footprint}$
$   \end{figure}$
$We describe the data we use in this section (Section~\ref{sec:data}), how the {\ha} and {\pa} emission-line maps are generated (Section~\ref{sec:maps}) and how the final sample is selected (Section~\ref{sec:sample_selection}).$
$\subsection{Data and data processing}$
$\label{sec:data}$
$We use data from the {\jwst} Cycle 1 programs NGDEEP and FRESCO. NGDEEP is a large treasury survey with 122.6 hours of NIRISS F115W, F150W, and F200W imaging and slitless spectroscopy observations centered on the HUDF and 96.4 hours of parallel NIRCam imaging of the HUDF-Par2 field: the deepest {\hst} ACS F814W field on the sky. FRESCO is a medium survey with 53.8 hours of F444W NIRCam imaging and slitless spectroscopy observations of the GOODS-S and GOODS-N CANDELS fields. In this work, we use the NIRISS data from NGDEEP and the GOODS-S NIRCam data from FRESCO, that overlaps over the HUDF (Figure~\ref{fig:footprint}). For more details on the NGDEEP and FRESCO surveys, we refer the reader to [Bagley, Pirzkal and Finkelstein (2024)]() and [Oesch, Brammer and Naidu (2023)](), respectively.$
$The Grism Redshift and Line Analysis Software (\texttt{grizli};  (Grizli2022) ) was used to process the imaging and slitless spectroscopy together. \texttt{grizli} downloads and pre-processes the raw data from the Mikulski Archive for Space Telescopes (MAST). A basis set of template flexible stellar population synthesis models (\texttt{FSPS};  (Conroy2009, Conroy2010) ) is projected to the pixel grid of the 2D grism exposures using the spatial morphology from the imaging. Observed spectra are then fit with 2D template spectra with non-negative least squares. To break redshift degeneracies, a line complex template is used and the grism redshift is taken to be where the \chi^{2} is minimised across the grid of trial redshifts input by the user.$
$\subsection{Emission-line maps}$
$\label{sec:maps}$
$The \sim2-4 times smaller bandwidths of the NIRISS filters and the order of magnitude lower resolving power of the NIRISS grisms compared to F444W NIRCam mean that the NGDEEP NIRISS slitless spectra are significantly shorter than the FRESCO NIRCam slitless spectra. The standard \texttt{grizli} approach of forward-modelling the direct imaging to construct a contamination model for each 2D grism exposure is therefore computationally feasible for NIRISS grism observations and is used to process the NGDEEP NIRISS observations ( ([Shen, Papovich and Matharu 2024]())  and references therein). In the \texttt{grizli}-processing for the FRESCO observations, grism spectra are instead median-filtered to remove the continuum ( ([Matharu, Nelson and Brammer 2024]())  and references therein). Emission-line maps are created by inputting the wavelength of the line in the \texttt{grizli}-extraction process, which creates continuum-subtracted narrow-band maps given a specific wavelength.$
$\subsection{Sample selection}$
$\label{sec:sample_selection}$
$The FRESCO grism spectra and \texttt{grizli} data products are visually inspected to verify {\pa} detections by the \texttt{grizli} software, as follows.$
$Sources with {\pa} signal-to-noise ratio, \mbox{SNR~>3} are initially selected from the \texttt{grizli} catalog for visual inspection. Two inspectors independently assess each using the \texttt{SpecVizitor} software\footnote{\href{https://github.com/ivkram/specvizitor}{https://github.com/ivkram/specvizitor}}, classifying them into five categories based on line fit quality and redshift probability distribution: (1) contaminated, (2) no lines, (3) uncertain line fit quality, (4) robust single-line fit with consistent F444W morphology, and (5) robust multiple line fits. Sources flagged as robust (quality 4 or 5) by both reviewers are labeled "robust"; those flagged by only one are "semi-robust". A weighting scheme accounts for inspector bias where necessary. For this study, we select robust (\sim30\%) and semi-robust (\sim 40\%) sources, further limited to those with {\pa} \mbox{SNR~>5}, yielding a final catalog of 508 robust {\pa} detections in GOODS-S. These detections are cross-matched with the 1153 {\ha} detections in the NGDEEP \texttt{grizli} extraction, yielding 43 matches. The NGDEEP grism spectra and \texttt{grizli} data products for this sample are visually inspected alongside the FRESCO \texttt{grizli} data products to verify the {\ha} detections and the quality of both the {\ha} and {\pa} emission-line maps. 35 galaxies are identified with good quality {\ha} and {\pa} emission-line maps. For more details on the quality criteria for emission-line maps, we refer the reader to Section 2.3 of [Matharu, Nelson and Brammer (2024)]().$
$\subsubsection{Stellar masses and star formation rates}$
$\label{sec:masses_sfms}$
$   \begin{figure}$
$   \centering$
$   \includegraphics[width=\columnwidth]{SFMS.pdf}$
$      \caption{Star formation main sequence of our sample measured using SED fitting (see Section~\ref{sec:masses_sfms}). The shaded grey regions delineate our stellar mass bins for the stacking analysis. SFRs and the [Popesso, Concas and Cresci (2023)]() SFMS include dust corrections. The dotted-line indicates extrapolation. The brown line and orange region delineate the FRESCO {\pa} SFMS from [Liu, Morishita and Kodama (2024)]() and [Neufeld and Asali (2024)]() respectively.}$
$         \label{fig:SFMS}$
$   \end{figure}$
$The Spectral Energy Distribution (SED) fitting code CIGALE  ([Boquien, Burgarella and Roehlly 2019](), Yang2020)  is used to derive stellar masses and SFRs for our galaxies in the NGDEEP NIRISS field. CANDELS  (Koekemoer2011, Grogin2011)  and JADES DR2  ([Eisenstein, Willott and Alberts 2023](), [Rieke, Robertson and Tacchella 2023](), [Williams, Tacchella and Maseda 2023]())  photometry spanning 0.43-4.8 \mum is used with the grism redshifts determined from the \texttt{grizli} NGDEEP extraction. The Calzetti1999 and [Cardelli, Clayton and Mathis (1989)]() dust laws with R_{V}=3.1 are used for attenuating the stellar continuum and emission-lines, respectively. Three Active Galactic Nuclei (AGNs) are identified and removed from the sample by cross-matching to [Lyu, et. al (2022)]() with a 1 arcsecond radius. AGN activity can lead to {\ha} and {\pa} emission. Because we are only interested in studying star formation traced by {\ha} and {\pa}, we remove AGN from our sample that would otherwise complicate the physical interpretation of our results. We then remove one galaxy from our sample for which we are unable to derive a stellar mass due to unavailable photometry. This leaves us with 31 galaxies that constitute our final sample. For full details on the SED fitting procedure -- including a comparison of {\ha} versus SED based SFRs -- we refer the reader to [Shen, Papovich and Matharu (2025)](). The star formation main sequence (SFMS) of our final sample is shown in Figure~\ref{fig:SFMS} along with the three stellar mass bins (grey shaded regions) we divide our sample into for forthcoming analysis. The majority of the sample lies along the literature SFMS from [Popesso, Concas and Cresci (2023)]() for the median redshift of our sample, suggesting we are primarily studying typical main sequence galaxies at this redshift.$
$\section{Methodology}$
$\label{sec:methodology}$
$In this section we describe how we prepare our data to enable the first direct comparison of {\ha} and {\pa} spatial profiles in the same galaxies at 1<z<1.8.$
$\subsection{Point-spread function matching}$
$\label{sec:PSFs}$
$   \begin{figure}$
$   \centering$
$   \includegraphics[width=\columnwidth]{PSF_profiles_Jy_smaller.pdf}$
$      \caption{PSF surface brightness profiles using the same star in the F150W NGDEEP NIRISS and F444W FRESCO NIRCam imaging. The thin pink line shows the PSF profile after PSF-matching (see Section~\ref{sec:PSFs}) the FRESCO PSF to the NGDEEP PSF. The amplitude (top panel) of the PSF is maintained and its shape (bottom panel) is appropriately altered.}$
$         \label{fig:PSFs}$
$   \end{figure}$
$The Point Spread Function (PSF) of an instrument describes how light spreads on a detector for a point source and so newcommandines the resolution limit of the detector. The {\ha} and {\pa} emission-line maps used in our study are obtained from slitless spectroscopy taken with two different instruments on-board JWST, which have different PSFs. Since the primary science goal of our study requires us to new a direct comparison of {\ha} and {\pa} spatial profiles in the same galaxies at these redshifts for the first time, the difference in PSFs needs to be accounted for. The PSF of a detector can be measured using observations of a star. The gold and blue thick lines in Figure~\ref{fig:PSFs} show the surface brightness profiles of the same star in the NGDEEP and FRESCO \texttt{grizli} direct image thumbnails which both have the same pixel scale and orientation. Not only do the amplitudes differ (top panel), but so do the shapes, more explicitly shown in the bottom panel of Figure~\ref{fig:PSFs}, where the profiles have been peak-normalised.$
$To address this problem, we PSF-match the higher resolution FRESCO observations to the lower resolution NGDEEP observations. This is done by supplying \texttt{pypher}  ([ and Boucaud 2016]())  with the direct image thumbnails of the two stars in the same units to generate a matching kernel between the FRESCO and NGDEEP observations. The \texttt{astropy} package  (TheAstropyCollaboration2018)  is then used to convolve the FRESCO F444W imaging and {\pa} maps with their respective matching kernels. The pink lines in Figure~\ref{fig:PSFs} show the result of this convolution on the FRESCO PSF profile. The amplitude of the FRESCO PSF is retained (top panel) while the shape is appropriately altered to match that of the NGDEEP PSF (bottom panel).$
$\subsection{Stacking}$
$\label{sec:stacking}$
$   \begin{figure}$
$   \centering$
$   \includegraphics[width=\columnwidth]{stacks_ergs_vertical.pdf}$
$      \caption{{\ha} (first column) and {\pa}-to-NGDEEP convolved (second column) stacks for our sample in three stellar mass bins (rows). The number of galaxies is shown in the bottom left corner of each {\ha} stack thumbnail. Each thumbnail is 60\times60 pixels with 1 pixel = 0.05^{\prime\prime}. Emission-line maps are generated such that the spectral axis is along the x-direction, while the y-direction is the spatial axis (see Section~\ref{sec:profiles}).}$
$         \label{fig:stacks}$
$   \end{figure}$
$To boost the signal-to-noise ratio as a function of galactocentric radius, we stack the {\ha} and {\pa} emission-line maps along with the direct imaging in three bins of stellar mass for our sample: {7.7\leqslant\mathrm{Log}(M_{*}/\mathrm{M}_{\odot})<8.8}, {8.8\leqslant\mathrm{Log}(M_{*}/\mathrm{M}_{\odot})<9.9} and {9.9\leqslant\mathrm{Log}(M_{*}/\mathrm{M}_{\odot})<11.0}. Whilst {\ha} is predominantly detected within the F150W filter wavelength range for our sample, there are a few cases where it is detected in the F200W wavelength range. We therefore use the NGDEEP F150W and F200W \texttt{grizli}-generated thumbnails of the star used for the PSF matching (Section~\ref{sec:PSFs}) between the two datasets to construct PSF stacks for our NGDEEP continuum and {\ha} stacks. Matching kernels are then created between the FRESCO PSF in F444W to these PSF stacks that are used to PSF-match (see Section~\ref{sec:PSFs}) the F444W continuum and {\pa} stacks to the NGDEEP stacks. Full details of our stacking process can be found in [Matharu, Papovich and Simons (2022)](). In summary, neighbouring sources are masked in both the direct image and emission-line map thumbnails using the \texttt{grizli}-generated segmentation map thumbnails for each galaxy. Pixels are weighted using the \texttt{grizli}-generated inverse variance map thumbnails and each galaxy is weighted by its total flux in the corresponding direct image filter such that no single bright galaxy dominates the stack. Direct image and emission-line map thumbnails are summed and exposure-corrected using the sum of their inverse variance maps. Each pixel is equal to 0.05 arcseconds and the dimensions of all thumbnails and the final stacks are 160\times160~pixels. Zoomed-in regions of our final {\ha} and PSF-matched {\pa} stacks can be seen in Figure~\ref{fig:stacks}.$
$\subsection{Surface brightness profiles}$
$\label{sec:profiles}$
$Due to the high spectral resolution of the NIRCam grism, maximal spatial information from the {\pa} emission-line maps can only be obtained along the cross-dispersion axis, hereafter the "spatial axis" (see Figure~\ref{fig:stacks} and  ([Matharu, Nelson and Brammer 2024]())  for more details). The FRESCO observations were taken at a position angle (PA) of zero (see Figure~\ref{fig:footprint}), leading to the spatial axis being along the vertical axis of all \texttt{grizli}-processed imaging, grism spectroscopy and emission-line maps. The NGDEEP \texttt{grizli} data processing was run with a rotation such that all final data products have PA=0. Therefore, the spatial axis of the FRESCO emission-line maps and imaging directly correspond to$
$the vertical axis of the NGDEEP emission-line maps and imaging. We therefore measure surface brightness profiles along the central vertical strip of all our stacks with \texttt{MAGPIE}\footnote{https://github.com/knaidoo29/magpie/}. These profiles form the basis of our analysis which we present in Section~\ref{sec:results}.$
$\section{Results}$
$\label{sec:results}$
$   \begin{figure*}$
$   \centering$
$   \includegraphics[width=\textwidth]{profiles_norm_new.pdf}$
$      \caption{Peak-normalised {\ha} (green) and {\pa} (turquoise) surface brightness profiles for our stacks shown in Figure~\ref{fig:stacks}. Both profiles are more extended than their PSFs, demonstrating they are well-resolved. At the lowest and highest masses, the {\ha} emission is more extended than the {\pa} emission, whereas their spatial profiles are similar for our middle stellar mass bin, at least out to \sim2 kiloparsecs after which the {\pa} emission is more extended than the {\ha} emission.}$
$         \label{fig:profiles_norm}$
$   \end{figure*}$
$Figure~\ref{fig:profiles_norm} shows the peak-normalised surface brightness profiles for the {\ha}, {\pa} and PSF stacks along the central vertical strip of the spatial axis. The emission-line map profiles are always more extended than their PSF profiles, demonstrating they are all spatially resolved by JWST NIRCam and NIRISS. For our lowest and highest mass bins, the {\ha} emission is more extended than the {\pa} emission. Whereas for our middle mass bin, {\ha} and {\pa} have similar spatial profiles out to \sim2~kpc after which {\pa} emission is more extended than the {\ha} emission out to 3~kpc.$
$   \begin{figure*}$
$   \centering$
$   \includegraphics[width=\textwidth]{EW_profiles.pdf}$
$      \caption{{\pa} (turquoise) and {\ha} (green) equivalent width (EW) profiles for our stacks with their Kendall's Tau (\tau) correlation statistic. All EW profiles display a positive radial gradient, demonstrating star-forming galaxies at these redshifts and stellar masses are growing inside-out {\it both} via dust-obscured ({\pa}) and unobscured ({\ha}) star formation.}$
$         \label{fig:profiles}$
$   \end{figure*}$
$   \begin{figure*}$
$   \centering$
$   \includegraphics[width=\textwidth]{profiles_linefluxes.pdf}$
$      \caption{Surface brightness profiles for our {\ha} (green) and {\pa} (turquoise) stacks (top row) with their ratios (bottom row) and Kendall's Tau (\tau) correlation statistic. The horizontal dotted line shows the expected {\pa}/{\ha} line ratio for Case B recombination, T_{e}=10,000~K and n_{e}=100~\mathrm{cm}^{-3} based on photoionisation modelling with \texttt{CLOUDY}  ([Ferland, Chatzikos and Guzmán 2017]()) . Declining {\pa}/{\ha} with galactocentric radius in our highest and lowest mass bins highlights the additional, dust-obscured star formation {\pa} is able to trace closer to galactic centers that decreases towards the outskirts of these galaxies.}$
$         \label{fig:linefluxes}$
$   \end{figure*}$
$We show the quotient of the emission-line map profiles with their respective continuum profiles in Figure~\ref{fig:profiles}. Known as the equivalent width (EW), this quantity news an absolute measurement for the strength of the emission line. We use the Kendall's Tau (\tau) statistic to measure the correlation between equivalent width and radius. Errors on \tau are computed using Monte Carlo sampling by drawing 1000 random samples of each line profile and their errors from a normal distribution with a standard deviation equal to the measurement errors. The error on \tau is then the standard deviation of the 1000 \tau measurements obtained. Both EW({\ha}) and EW({\pa}) have positive profile gradients as a function of radius for all stellar mass bins. EW({\pa}) always has a higher normalisation than EW({\ha}).$
$Figure~\ref{fig:linefluxes} shows the emission line flux profiles for our {\ha} and {\pa} stacks and their ratios. As demonstrated by the \tau statistic, our {\pa}/{\ha} profiles have negative gradients with increasing radius for the lowest and highest stellar mass bins, but a mildly positive gradient with increasing radius for our middle stellar mass bin.$
$In the next section, we will discuss the physical interpretation of our results.$
$\section{Discussion}$
$\label{sec:discussion}$
$The most massive and short-lived (\sim10~Myr) O-type stars emit the strongest ultraviolet (UV) radiation, capable of stripping hydrogen atoms of their electrons. The recombination of these electrons with ionised hydrogen atoms further from the centre of these stars leads to the emission of hydrogen recombination lines such as {\ha} in the rest-frame optical and {\pa} in the rest-frame near-infrared (NIR). Whilst the {\ha} emission line is intrinsically stronger than the {\pa} emission line -- and therefore more than eight times brighter (Case B recombination, T_{e}=10^{4}~K) -- it is more susceptible to dust attenuation. Therefore, {\ha} is often regarded as a good tracer of unobscured ongoing star formation only. {\pa} is dubbed a "gold standard" tracer of ongoing star formation due to its capability in tracing more heavily dust-obscured ongoing star formation  (Kennicutt2012) .$
$\subsection{Galaxy growth}$
$\label{sec:galaxy_growth}$
$The equivalent width (EW) profiles of each of these emission lines (Figure~\ref{fig:profiles}) measure their strength with respect to their underlying continuum as a function of galactocentric radius. The normalisation of both EW profiles fall with increasing stellar mass, consistent with a fall in sSFRs ( ([Popesso, Concas and Cresci 2023]())  and references therein) and the increasing contribution of older stellar populations to the stellar continuum both in the optical and NIR. In general, all EW profiles are positive for all three stellar mass bins, providing direct evidence for the inside-out growth of galaxies via star formation after cosmic noon, confirmed here for the first time with both {\ha} and {\pa} (see also  ([Liu, Morishita and Kodama 2024]())  for {\pa} versus continuum size analysis).$
$Areas that are often missed in EW({\ha}) profiles are optically thick regions at the rest-frame wavelength of {\ha} (6565{Å}) and/or regions that have higher nebular dust attenuation which reduce the {\ha} line flux more relative to the stellar attenuation reducing the continuum flux. In the first case, both the {\ha} line flux and continuum is affected. The second case could be the case for star-forming regions (e.g.  (Calzetti1999, [ and é 2009](), Yoshikawa2010, [Mancini and Renzini 2011](), Wuyts2011, [Wuyts and Nelson 2013](),  Kashino2013, [Kreckel, Groves and Schinnerer 2013](), [Price, Kriek and Brammer 2014](), Reddy2015, Bassett2017,  Theios2019, Koyama2019a, Greener2020,  Wilman2020, Rodriguez-Munoz2021) ) and leads to an underestimation in EW({\ha}). In the next section, we therefore discuss a cleaner approach to comparing unobscured versus obscured star formation using our measurements.$
$\subsection{Unobscured vs. obscured star formation}$
$It is impossible to gain deeper insight into spatially resolved unobscured versus obscured star formation by directly comparing {\ha} and {\pa} EW profiles. This is because of complications introduced by differential dust attenuation of the {\ha} line and the continuum at 6565{Å} (see Section~\ref{sec:galaxy_growth}) as well as differing relative contributions of different stellar populations to the continuum in the rest-frame optical versus the NIR. We therefore instead directly compare the {\ha} and {\pa} line profiles with their ratios in Figure~\ref{fig:linefluxes}. On average, line flux increases with stellar mass inline with increasing SFRs along the main sequence (see Figure~\ref{fig:SFMS}). In the lowest and highest mass bins, the {\pa}/{\ha} profiles have declining trends with radius from the center towards the expected value of {\pa}/{\ha} in the absence of dust attenuation (horizontal dotted line, Case B recombination, T_{e}=10,000~K and n_{e}=100~\mathrm{cm}^{-3} based on photoionisation modelling with \texttt{CLOUDY}  ([Ferland, Chatzikos and Guzmán 2017]()) ). In our lowest mass bin, we are predominantly probing galaxies with high sSFRs and in our highest mass bin, those with the highest SFRs in our sample (see Figure~\ref{fig:SFMS}). This trend in the {\pa}/{\ha} profiles suggests that the dust in the central regions of these galaxies is preferentially attenuating the {\ha} flux relative to the {\pa} flux. Our result at the highest stellar masses is consistent with those of many ALMA studies (see discussion in  ([Shen, Papovich and Yang 2023]()) ) and recent results found using JWST MIRI imaging by [Shen, Papovich and Yang (2023)]() and [Magnelli, Gómez-Guijarro and Elbaz (2023)](). The rate of galaxy growth via star formation with radius measured using {\ha} emission alone in the central regions (\sim1.5~kpc) of such galaxies could therefore be overestimated.$
$Contrastingly, the relatively flat {\pa}/{\ha} line profile of the main sequence galaxies in our middle mass bin suggests that not all main sequence galaxies at cosmic noon that are growing inside-out via star formation have centrally concentrated dust attenuation. This more complicated history of star formation for the central and outer regions of galaxies is supported by the recent results of [Shen, Papovich and Matharu (2024)]() who examined spatially resolved EW({\ha}), sSFR and age for 19 main sequence galaxies at 0.6<z<2.2 of comparable stellar masses ({9\leqslant\mathrm{Log}(M_{*}/\mathrm{M}_{\odot})<11}) to our sample in the same (NGDEEP NIRISS) field (see also consistent results in  ([Matharu, Muzzin and Sarrouh 2023](), [Liu, Morishita and Kodama 2023](), [Liu, Morishita and Kodama 2024](), [Maheson, Tacchella and Belli 2025]()) ). Whilst the majority (84\%) of their galaxies have positive EW({\ha}) profiles supporting the inside-out growth scenario, the sSFR and star formation history (SFH) profiles suggest at least one rapid star formation episode is responsible for forming the bulge, but smoothly varying SFHs for the disk suggest longer timescale inside-out growth. Our results along with those of [Shen, Papovich and Matharu (2024)]() put forth a more complex picture for spatially resolved star formation at and around cosmic noon. Large sample resolved studies of individual galaxies at cosmic noon across the main sequence are now required to understand the intrinsic scatter in how star formation progresses in galaxies.$
$\section{Summary}$
$\label{sec:summary}$
$By processing overlapping JWST NGDEEP NIRISS and FRESCO NIRCam slitless spectroscopy in the HUDF, we have newd the first direct comparison of spatially resolved unobscured (traced by {\ha}) and dust-obscured (traced by {\pa}) ongoing star formation in 31 galaxies at 1<z<1.8 (Section~\ref{sec:results}). Our main conclusions are listed below.$
$\begin{enumerate}$
$    \item Positive EW({\pa}) and EW({\ha}) profiles demonstrate that main sequence galaxies just after cosmic noon are growing inside-out {\it both} via dust-obscured and unobscured star formation.$
$    \item At the highest stellar masses and sSFRs, {\pa}/{\ha} falls with increasing galactocentric radius, likely tracing decreasing dust-obscured star formation relative to unobscured star formation towards larger radii.$
$    \item The relatively flat {\pa}/{\ha} line profile for our stack of galaxies with stellar masses 8.8\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<9.9 suggests centrally concentrated dust attenuation is not ubiquitous across the main sequence just after cosmic noon in galaxies growing inside-out via star formation.$
$\end{enumerate}$
$Our work puts forth a proof-of-concept for using NIRISS and NIRCam slitless spectroscopy together to accomplish more detailed resolved studies of high redshift galaxies. Together with other similar works in the literature, this work motivates a strong case for detailed, resolved studies of large samples of individual galaxies across the main sequence at cosmic noon to understand the intrinsic scatter in spatially resolved star formation.$
$\begin{acknowledgements}$
$      JM is grateful to the Max Planck Society for the MPIA Prize Fellowship and to the Cosmic Dawn Center for the DAWN Fellowship. This work is based on observations made with the NASA/ESA/CSA James Webb Space Telescope. The raw data were obtained from the Mikulski Archive for Space Telescopes at the Space Telescope Science Institute, which is operated by the Association of Universities for Research in Astronomy, Inc., under NASA contract NAS 5-03127 for \textit{JWST}. These observations are associated with JWST Cycle 1 GO programs \#1895 and \#2079. Support for both programs was newd by NASA through a grant from the Space Telescope Science Institute, which is operated by the Associations of Universities for Research in Astronomy, Incorporated, under NASA contract NAS5-26555.$
$      The Cosmic Dawn Center (DAWN) is funded by the Danish National Research Foundation under grant DNRF140.$
$      This work has received funding from the Swiss State Secretariat for Education, Research and Innovation (SERI) under contract number MB22.00072, as well as from the Swiss National Science Foundation (SNSF) through project grant 200020\_207349.$
$      This work benefited from support from the George P. and Cynthia Woods Mitchell Institute for Fundamental Physics and Astronomy at Texas A\&M University. CP thanks Marsha and Ralph Schilling for generous support of this research.$
$\end{acknowledgements}$
$\bibliography{library_grizli2} $
$\bibliographystyle{aa}$
$\end{document}}$</div>



<div id="title">

# A first look at a complete view of spatially resolved star formation at $1<z<1.8$ with JWST NGDEEP+FRESCO slitless spectroscopy

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.15792-b31b1b.svg)](https://arxiv.org/abs/2511.15792)<mark>Appeared on: 2025-11-21</mark> -  _7 pages, 7 figures, to be submitted to A&A_

</div>
<div id="authors">

<mark>J. Matharu</mark>, et al.

</div>
<div id="abstract">

**Abstract:** The previously inaccessible star formation tracer Paschen-Alpha ( ${\pa}$ ) can now be spatially resolved by JWST NIRCam slitless spectroscopy in distant galaxies up to cosmic noon. In the first study of its kind, we combine JWST NGDEEP NIRISS and FRESCO NIRCam slitless spectroscopy to provide the first direct comparison of spatially resolved dust-obscured (traced by ${\pa}$ ) versus unobscured (traced by ${\ha}$ ) star formation across the main sequence. We stack ${\pa}$ and ${\ha}$ emission-line maps,  along with stellar continuum images at both wavelengths of 31 galaxies at $1<z<1.8$ in three bins of stellar mass spanning $7.7\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<11$ . Surface brightness profiles are measured and equivalent width (EW) profiles computed. Increasing ${\pa}$ and ${\ha}$ EW profiles with galactocentric radius across all stellar masses probed provide direct evidence for the inside-out growth of galaxies both via dust-obscured and unobscured star formation for the first time. For galaxies predominantly on the main sequence ( $\mathrm{log}(M_{*}/\mathrm{M}_{\odot})\geqslant8.8$ ), a weakly positive ( $0.1\pm0.1$ ) ${\pa}$ / ${\ha}$ line profile gradient as a function of radius is found at $8.8\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<9.9$ with a negative ( $-0.4\pm0.1$ ) ${\pa}$ / ${\ha}$ line profile gradient as a function of radius found at the highest stellar masses of $9.9\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<11.0$ . Low mass galaxies ( $7.7\leqslant\mathrm{log}(M_{*}/\mathrm{M}_{\odot})<8.8$ ) with predominantly high specific star formation rates (sSFRs) relative to the main sequence are also found to have a negative ( $-0.5\pm0.1$ ) ${\pa}$ / ${\ha}$ line profile gradient as a function of radius.   Our results demonstrate that while inside-out growth via star formation is ubiquitous across the main sequence just after cosmic noon, centrally concentrated dust attenuation is not. Along with other recent work in the literature, our findings now motivate future studies of resolved SFR and dust attenuation profiles in large samples of individual cosmic noon galaxies across the main sequence, to understand the intrinsic scatter in spatially resolved star formation.

</div>

<div id="div_fig1">

<img src="tmp_2511.15792/./SFMS.png" alt="Fig2" width="100%"/>

**Figure 2. -** Star formation main sequence of our sample measured using SED fitting (see Section \ref{sec:masses_sfms}). The shaded grey regions delineate our stellar mass bins for the stacking analysis. SFRs and the [Popesso, Concas and Cresci (2023)]() SFMS include dust corrections. The dotted-line indicates extrapolation. The brown line and orange region delineate the FRESCO {$\pa$} SFMS from [Liu, Morishita and Kodama (2024)]() and [Neufeld and Asali (2024)]() respectively. (*fig:SFMS*)

</div>
<div id="div_fig2">

<img src="tmp_2511.15792/./PSF_profiles_Jy_smaller.png" alt="Fig3" width="100%"/>

**Figure 3. -** PSF surface brightness profiles using the same star in the F150W NGDEEP NIRISS and F444W FRESCO NIRCam imaging. The thin pink line shows the PSF profile after PSF-matching (see Section \ref{sec:PSFs}) the FRESCO PSF to the NGDEEP PSF. The amplitude (top panel) of the PSF is maintained and its shape (bottom panel) is appropriately altered. (*fig:PSFs*)

</div>
<div id="div_fig3">

<img src="tmp_2511.15792/./stacks_ergs_vertical.png" alt="Fig4" width="100%"/>

**Figure 4. -** {$\ha$}(first column) and {$\pa$}-to-NGDEEP convolved (second column) stacks for our sample in three stellar mass bins (rows). The number of galaxies is shown in the bottom left corner of each {$\ha$} stack thumbnail. Each thumbnail is $60\times60$ pixels with 1 pixel = 0.05$^{\prime\prime}$. Emission-line maps are generated such that the spectral axis is along the $x$-direction, while the $y$-direction is the spatial axis (see Section \ref{sec:profiles}). (*fig:stacks*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.15792"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

131  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

16  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
